In [1]:
%reload_ext autoreload
%autoreload 2
# %matplotlib qt

from time import time
from tqdm import tqdm
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

from cellector.io import create_from_suite2p, save_selection
from cellector.roi_processor import RoiProcessor
from cellector.gui import SelectionGUI

In [7]:
s2p_dir = Path(r"C:\Users\Andrew\Documents\test_cellector\suite2p")
roi_processor = create_from_suite2p(s2p_dir, use_redcell=True, clear_existing=True)

In [6]:
gui = SelectionGUI(roi_processor)

red_s2p [2.5576864e-07, 0.9875573] <class 'list'>
[None, 0.9875573] [False, True]
phase_corr [-0.12295520518103062, 0.4105598307681927] <class 'list'>
[None, 0.4105598307681927] [False, True]
dot_product [-66807.79264980495, 203016.05645847096] <class 'list'>
[-66807.79264980495, 203016.05645847096] [True, True]
corr_coef [-0.7663644687328549, 0.985466595444871] <class 'list'>
[-0.7663644687328549, 0.985466595444871] [True, True]
in_vs_out [0.0948887, 0.47972274] <class 'list'>
[0.0948887, 0.47972274] [True, True]


In [36]:
gui.feature_cutoffs

{'red_s2p': array([2.5576864e-07, 9.8755729e-01], dtype=float32),
 'phase_corr': array([0.01274831, 0.41055983]),
 'dot_product': array([ 22337.56887429, 203016.05645847]),
 'corr_coef': array([-0.76636447,  0.9854666 ]),
 'in_vs_out': array([0.0948887 , 0.47972274], dtype=float32)}

In [5]:
check = np.load(s2p_dir / "plane0" / "cellector" / "phase_corr_criteria.npy")
print(check.shape, check)

(2,) [       nan 0.41055983]


red_s2p [2.5576864e-07 9.8755729e-01]
[      nan 0.9875573] [False, True]
phase_corr [0.01274831 0.41055983]
[       nan 0.41055983] [False, True]
dot_product [ 22337.56887429 203016.05645847]
[22337.56887429            nan] [True, False]
corr_coef [-0.76636447  0.9854666 ]
[      nan 0.9854666] [False, True]
in_vs_out [0.0948887  0.47972274]
[       nan 0.47972274] [False, True]


In [18]:
manual_selection = np.stack((gui.manual_label, gui.manual_label_active)).T
save_selection(gui.roi_processor, gui.idx_selected, gui.feature_cutoffs, manual_selection)
gui.update_text("Selection saved!")

In [2]:
# Next step: 
# 1. Implement a saving method (and maybe a two way system for creating a red cell object ? from suite2p or from scratch)
# 2. Visualization methods for optimizing the hyperparameters for preprocessing
# 3. Implement an optimization method for phase_corr_eps? 
#    - Might just be as simple as the user inputing the maximum size ROI, and then measuring
#      the average power for higher frequencies...

In [5]:
gui = SelectionGUI(roi_processor)